# Python interaction

In this page I will show you how to deploy and interact with a Postgres database in a Docker container.

**Note** `psycopg2` is a central library for organising Python<->Postgres interaction, developers distribute it uncompiled for optimisation purposes, but if you just want to install and use it `pip3 install psycopg2-binary`.

## Python from container

For me it's common task to build interaption with container which handle python program.

So in this section I want to show you how to build such an interaction.

### Python image

You need to create a Python image. The following cell describes the docker file that's used to create the image.

In [1]:
%%writefile python_interaction/python_container/dockerfile
FROM python:3.11.4
WORKDIR program
COPY script.py script.py
COPY requirements.txt requirements.txt
RUN pip3 install -r requirements.txt

Overwriting python_interaction/python_container/dockerfile


The next cell describes the `requirements.txt` we'll need for this container.

In [2]:
%%writefile python_interaction/python_container/requirements.txt
numpy==1.24.2
psycopg2-binary==2.9.6

Overwriting python_interaction/python_container/requirements.txt


Finally, we build the image.

In [3]:
%%bash
docker build -t pg_example_python python_interaction/python_container &> /dev/null

## Start containers

We need to create at least one table in our postgres database. So in the following cell we store the script that will be used to initialise the database.

In [7]:
%%writefile python_interaction/create_table.sql
CREATE TABLE main_table(
    id TEXT NOT NULL,
    text TEXT NOT NULL
);

Overwriting python_interaction/create_table.sql


Next, start the `python` and `postgres` containers.

You should share the same net between containers, in the following example which is completed by `test_project_net`.

In [13]:
%%bash
# network
docker network create test_project_net &> /dev/null
# postgres
docker run --rm -d\
    --name pg_example_posgres_cont\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    --net=test_project_net\
    -v ./python_interaction/create_table.sql:/docker-entrypoint-initdb.d/create_table.sql\
    postgres:15.4 &> /dev/null
# python
docker run --rm -itd\
    --name pg_example_python_cont\
    --net=test_project_net\
    pg_example_python &> /dev/null

### Executing python

In the next cell, we run the Python script in the Python container. The script simply adds some random values to the `main_table` from the **ohter container**. The message "Adding records is done!" indicates that the Python program was executed normally.

In [15]:
%%bash
docker exec pg_example_python_cont bash -c "python3 script.py"

Adding records is done!


### Check the table

So now, to make sure we have done everything right, let us select values from the created table.

In [16]:
%%bash
docker exec pg_example_posgres_cont bash -c \
"psql -U docker_app -d docker_app_db -c \"SELECT * FROM main_table;\""

 id |         text         
----+----------------------
 0  | japnymalatjibwyyhrrg
 1  | cbbweteuzciiqkfeosan
 2  | pceqighcvhuhsgkzkyjm
 3  | ppnxmfwrhfxakxmrbgfa
 4  | rgcskpyszvvfgkeqlddg
 5  | hgstbkchoffmwzfzstzl
 6  | zppfchaejglgziuqsgfc
 7  | lbketfergbfpnqigunpi
 8  | vwloacncpjvjvpkcrsen
 9  | fdmykbfwrpatwgqieerk
 10 | aphatmxojirrgapuoyig
 11 | ngrltrqxulquwymuwjzt
 12 | ccutblvvmrzwgixpojlj
 13 | vrwzqmdxnlhwuwqtsxdd
 14 | cxngyvgptjeqtgoghgsj
 15 | knuspqeycfulmybbjwpz
 16 | bhhfyemeyustohgvvfzx
 17 | wpkrxlifxhkafoygxhyy
 18 | tbmofckmfqmncyuhwzlp
 19 | kpzirtepbdhjshzdpgfd
(20 rows)



### Stop containers

In [17]:
%%bash
docker stop pg_example_posgres_cont pg_example_python_cont &> /dev/null
docker network rm test_project_net &> /dev/null

### Clear python image

To avoid creating a lot of rubbish in docker images, you should delete the container created for this example.

In [31]:
%%bash
docker rmi pg_example_python &> /dev/null

## Python on host

Sometimes it's good to run database-related code from your local machine. So I show a way to build interaption between the database and the host program.

### Database container

Script that is used ot create table in database described in the following cell:

In [21]:
%%writefile python_interaction/create_table.sql
CREATE TABLE main_table(
    id TEXT NOT NULL,
    text TEXT NOT NULL
);

Overwriting python_interaction/create_table.sql


**Note** you need to specify the port to be referred to later in the Python program.

In [22]:
%%bash
docker run --rm -d\
    --name pg_example_posgres_cont\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    -p 5431:5432\
    -v ./python_interaction/create_table.sql:/docker-entrypoint-initdb.d/create_table.sql\
    postgres:15.4 &> /dev/null

### Python program

I'm just going to connect to the database from this notebook.

#### Establish conneciton

In the `psycopg2.connect` function, mention `port` used in postgres container creation and `localhost` argument for `host` parameter.

In [23]:
import psycopg2

conn = psycopg2.connect(
    port = "5431", # same as when creating a postgres container
    dbname = "docker_app_db",
    user = "docker_app",
    password = "docker_app",
    host= "localhost"
)

### Insert information

To understand that everything works insert a few lines into database.

In [24]:
import random
import string

cur = conn.cursor()
for i in range(20):
    text = ''.join(random.choices(string.ascii_lowercase, k=20))
    query = f"INSERT INTO main_table (id, text) VALUES ('{i}', '{text}');"
    cur.execute(query)
cur.close()

### Check the result

Query from python.

In [25]:
cur = conn.cursor()
cur.execute("SELECT * FROM main_table;")
for i in cur:
    print(i)
cur.close()

('0', 'wmrdtxsodxscjvhewqal')
('1', 'ufkuejcrfxftdnhdxrag')
('2', 'ptevrscclarwcrqbeebu')
('3', 'miusgnfiytlzliwkzfsw')
('4', 'mccrfcyzbobchnsctsoa')
('5', 'hvrcrkodrybhikjtxrzg')
('6', 'dywhqgbaqdgrndzyexsy')
('7', 'hokekkgjxrrmjmgdbjog')
('8', 'lkrgeqzubnmasudrsmwa')
('9', 'unnlpkbjbbgflgllypzp')
('10', 'yjxwtmrmiluhrfxemfbm')
('11', 'slewtzvcobmuvjtfjzuq')
('12', 'rxykfqyhymbxrjrrfslj')
('13', 'ttdkznjfzgsbhpatxexc')
('14', 'tlcgljqyykggziysejln')
('15', 'nexvhxibjodjgoqucffy')
('16', 'djfefydvsfzepcygfdke')
('17', 'byadocobydotccunhypu')
('18', 'oukrqsvgpsgjqdcuueax')
('19', 'qfnxjpfbmjgbpombtycu')


Query from container. 

**Note** Before quering from the container, you need to commit the changes from the connection.

In [27]:
conn.commit()

In [28]:
%%bash
docker exec pg_example_posgres_cont \
    psql --username docker_app --dbname docker_app_db -c 'SELECT * FROM main_table;'

 id |         text         
----+----------------------
 0  | wmrdtxsodxscjvhewqal
 1  | ufkuejcrfxftdnhdxrag
 2  | ptevrscclarwcrqbeebu
 3  | miusgnfiytlzliwkzfsw
 4  | mccrfcyzbobchnsctsoa
 5  | hvrcrkodrybhikjtxrzg
 6  | dywhqgbaqdgrndzyexsy
 7  | hokekkgjxrrmjmgdbjog
 8  | lkrgeqzubnmasudrsmwa
 9  | unnlpkbjbbgflgllypzp
 10 | yjxwtmrmiluhrfxemfbm
 11 | slewtzvcobmuvjtfjzuq
 12 | rxykfqyhymbxrjrrfslj
 13 | ttdkznjfzgsbhpatxexc
 14 | tlcgljqyykggziysejln
 15 | nexvhxibjodjgoqucffy
 16 | djfefydvsfzepcygfdke
 17 | byadocobydotccunhypu
 18 | oukrqsvgpsgjqdcuueax
 19 | qfnxjpfbmjgbpombtycu
(20 rows)



Close connection.

In [29]:
conn.close()

### Stop container

In [30]:
%%bash
docker stop pg_example_posgres_cont

pg_example_posgres_cont
